## RAI vs EUR GBP CHF AUD JPY (base USD) 

Read on-chain RAI data

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt

from web3 import Web3, HTTPProvider

from abis import LINK_ETH, LINK_ETH_ABI, LINK_ETH_FIRST_BLOCK
from abis import LINK_RAI, LINK_RAI_ABI, LINK_RAI_FIRST_BLOCK
from abis import ORACLE_RELAYER, ORACLE_RELAYER_ABI, ORACLE_RELAYER_FIRST_BLOCK

from mp import fetch, fetch_link_mp, fetch_rp

size = 15
params = {'legend.fontsize': 'large',
          'figure.figsize': (20,12),
          'axes.labelsize': size,
          'axes.titlesize': size,
          'xtick.labelsize': size*0.75,
          'ytick.labelsize': size*0.75,
          'axes.titlepad': 25}

plt.rcParams.update(params)

In [ ]:
# Need an archive node

ETH_RPC_URL=os.environ['ETH_RPC_URL']
web3 = Web3(HTTPProvider(endpoint_uri=ETH_RPC_URL, request_kwargs={"timeout": 10}))
print(ETH_RPC_URL)

In [ ]:
first_block = 13334360 # to be set to ~1 year before ExRates API start_date monthly, considering monthly output of data
interval = 6379 * 1 #number of blocks per day
num_blocks = None

latest_block = web3.eth.get_block("latest")['number']

if not num_blocks:
    last_block = latest_block - 12
else:
    last_block = min(latest_block - 12, first_block + num_blocks)

print(f"first block {first_block}, last_block {last_block}")
blocks = list(range(first_block, last_block + 1, interval))

In [ ]:
#start = time.time()
mp_results = fetch(fetch_link_mp, 8, LINK_RAI, LINK_RAI_ABI, ETH_RPC_URL, blocks=blocks)
#print(f"took {time.time() - start}"

In [ ]:
rp_results = fetch(fetch_rp, 8, ORACLE_RELAYER, ORACLE_RELAYER_ABI, ETH_RPC_URL, blocks=blocks)

In [ ]:
rp_df = pd.DataFrame(rp_results)
#rp_df.index
rp_df.head(10)

In [ ]:
mp_df=pd.DataFrame(mp_results)
mp_df.drop([3], axis=1, inplace=True)
mp_df.head(10)

In [ ]:
rp_df.index

In [ ]:
mp_df.index

In [ ]:
rai_df = pd.merge(rp_df,mp_df, left_on=[0], right_on=[0], how='outer').astype(object)
rai_df.columns=(['block','RAI_rp','RAI_mp','UnixTimestamp'])
rai_df

In [ ]:
rai_df['RAI_rp'] = rai_df['RAI_rp']*1e-27
rai_df['RAI_mp'] = rai_df['RAI_mp']*1e-8
rai_df.index = pd.to_datetime(rai_df['UnixTimestamp'], unit='s').dt.date
rai_final_df = rai_df[['RAI_rp','RAI_mp']]
rai_final_df.index.name = None

In [ ]:
#rai_final_df = rai_final_df.transpose()
rai_final_df

In [ ]:
from ExRates import ExRates_final_df

In [ ]:
ExRates_final_df = ExRates_final_df.transpose()
ExRates_final_df

In [ ]:
df = pd.merge(ExRates_final_df, rai_final_df,left_index=True, right_index=True)
#df.to_csv('RAIvsEURGBPCHFAUDJPYCNY.csv')
#pd.set_option('display.max_rows', 50)
df

In [ ]:
df.index

In [ ]:
first_date = min(df.index)
rai_rp_setpoint = df.loc[first_date, 'RAI_rp']
rai_mp_setpoint = df.loc[first_date, 'RAI_mp']
eur_setpoint = df.loc[first_date, 'EUR']
gbp_setpoint = df.loc[first_date, 'GBP']
chf_setpoint = df.loc[first_date, 'CHF']
aud_setpoint = df.loc[first_date, 'AUD']
cad_setpoint = df.loc[first_date, 'CAD']
jpy_setpoint = df.loc[first_date, 'JPY']
cny_setpoint = df.loc[first_date, 'CNY']
df['rai_rp_norm'] = (df['RAI_rp']/rai_rp_setpoint)
df['rai_mp_norm'] = (df['RAI_mp']/rai_mp_setpoint)
df['eur_norm'] = (df['EUR']/eur_setpoint)
df['gbp_norm'] = (df['GBP']/gbp_setpoint)
df['chf_norm'] = (df['CHF']/chf_setpoint)
df['aud_norm'] = (df['AUD']/aud_setpoint)
df['cad_norm'] = (df['CAD']/cad_setpoint)
df['jpy_norm'] = (df['JPY']/jpy_setpoint)
df['cny_norm'] = (df['CNY']/cny_setpoint)

In [ ]:
df


In [ ]:
# df['dates'] = df.index.astype(object)
# df

In [ ]:
fig, ax = plt.subplots(1,1)
ax.set_title('Major Currencies and RAI in relation to USD', size=20)
ax.plot(df['rai_rp_norm'], label = 'RAI_rp/USD- normalized', linewidth = 4, color = 'green')
ax.plot(df['rai_mp_norm'], label = 'RAI_mp/USD- normalized', linewidth = 3, alpha= 0.8, color = (0.2,0.7,0.3))
ax.plot(df['eur_norm'], linewidth= 2, alpha= 0.9, label='EUR/USD - normalized', color='black')
ax.plot(df['gbp_norm'], linewidth= 2, alpha= 0.9, label='GBP/USD - normalized', color='red')
ax.plot(df['chf_norm'], linewidth= 2, alpha= 0.9, label='CHF/USD - normalized', color='blue')
ax.plot(df['aud_norm'], linewidth= 2, alpha= 0.9, label='AUD/USD - normalized', color='orange')
ax.plot(df['cad_norm'], linewidth= 2, alpha= 0.9, label='CAD/USD - normalized', color='violet')
ax.plot(df['jpy_norm'], linewidth= 2, alpha= 0.9, label='JPY/USD - normalized', color='gray')
ax.plot(df['cny_norm'], linewidth= 2, alpha= 0.9, label='CNY/USD - normalized', color='brown')
ax.set_xlabel('Date')
# ax.set_ylabel('/USD (normalised)')
ax.legend()
ax.grid()
fig.set_figwidth(30)